# DAX-Projekt: Datenbeschaffung auf die andere Art

Nach dem Enron-Fall möchten wir uns als nächstes einem anderen Wirtschaftsthema widmen, nämlich der Deutschen Börse.

Die Beschaffungswege um an Daten heranzukommen sind immer noch die selbigen:

A) Die Daten liegen im Netz zum direkten Download bereit (wenn wir Glück haben sogar im kuratierten Zustand). Dabei können die Daten beispielhaft in folgenden Formaten sein:

    strukturiert: csv, xls, xlsx, parquet, orc
    semistrukturiert: html, json, yaml, xml, rdf, sql
    unstrukturiert: ppt, pptx, pdf, docx, zip, pst, png, img, mp3, avi, mp4

B) Die Daten werden über eine Schnittstelle (API) angeboten und können darüber abgegriffen werden

C) Die Daten, die uns interessieren sind nicht einfach herunterladbar und wir müssen sie von Webseiten aus der HTML-Repräsentation abgreifen (web scraping).

Wir möchten zu Beginn erstmal nur Informationen sammeln, nämlich welche Unternehmen derzeit überhaupt im DAX gelistet sind. Das finden wir zum Beispiel durch die entsprechende Seite von Wikipedia heraus *https://en.wikipedia.org/wiki/DAX*. Wenn man Daten von einer Webseite ziehen möchte, ist es oft eine gute Idee, sich erst mal anzusehen, wie die Seite aufgebaut ist. Klicke auf den Link und schau dir die Seite kurz an.

Die aktuellen Unternehmen sind unter der Überschrift **Components** in einer Tabelle gelistet. Diese werden wir später auslesen. 



---

# Aufgabe 1: Datenabzug über HTML

Zuerst verbinden wir uns mit der Webseite. Daten werden im Internet meist mit Hilfe von HTTPS (Hypertext Transfer Protocol Secure) übertragen. Das ist eine verschlüsselte Variante von HTTP ([Hyptertext Transfer Protocol](https://de.wikipedia.org/wiki/Hypertext_Transfer_Protocol)). Dabei wird durch den Browser eine Anfrage an den Server geschickt und dieser schickt eine Antwort zurück.

- Importiere das Modul `requests`
- Durch `requests` können wir die Anfrage auch ohne Browser an den Webserver schicken. Dazu stellt das Modul die Funktion `requests.get()` bereit. Speichere die Adresse der Wikipedia-Seite als *string* in der Variablen `website_url`. Überreiche diese Variable an `requests.get()` und speichere das Ergebnis als `response`. Drucke es anschließend aus.

Du solltest etwas wie `<Response [200]>` erhalten haben. Wir erinnern uns, es gibt fünf Zahlenräume für die Statuscodes:

| Zahlenraum | Bedeutung |
| ---------- | --------- |
| 1XX | Die Anfrage wurde erhalten (Information) |
| 2XX | Die Anfrage wurde erhalten und akzeptiert (Erfolgreiche Anfrage) |
| 3XX | Weitere Aktionen werden durchgeführt, um die Anfrage zu erfüllen (Umleitung) |
| 4XX | Die Anfrage kann nicht umgesetzt werden, was vermutlich am Client liegt (Client-Fehler) |
| 5XX | Die Anfrage kann nicht umgesetzt werden, was vermutlich am Server liegt (Server-Fehler) |

Die nächsten beiden Zahlen spezifizieren die Antwort. Eine umfangreiche Liste der Codes kannst du [hier](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes) finden.

Wir haben eine 200 erhalten. Dieser Code steht dafür, dass alles rund gelaufen ist. Weitere wichtige Codes sind folgende:
* 401 (der Zugriff auf die gesuchten Informationen wurde verweigert, evtl. wegen fehlender/fehlerhafter Authentifizierung)
* 404 (die gesuchten Informationen wurden nicht gefunden)
* 429 (zu viele Anfragen wurden in einem bestimmten Zeitfenster gestellt)

Schauen wir uns unsere Antwort nun etwas genauer an. Welchen Datentyp hat sie?

Neben dem Statuscode können wir weitere Informationen aus unserer Antwort ziehen. Drucke das Attribut `my_response.headers`.

Der Output ist auf den ersten Blick etwas unübersichtlich. Aber an den geschweiften Klammern erkennen wir, dass es sich vermutlich um ein *dictionary* handelt. Das stimmt so aber noch nicht ganz. Da die Headerdaten für die Kommunikation mittels HTTP als *case-insensitive* definiert wurden, gilt das auch für `my_response.headers`. Es ist also egal, ob die Buchstaben der *keys* groß oder klein sind.
`my_response.headers` enthält Metadaten zu der Antwort, die wir erhalten haben. Welche Werte stecken hinter den *keys* `'content-type'` und `'date'`? Gib sie aus.

Tipp: Wie eben erwähnt sind die *keys* hier, im Gegensatz zum normalen `dict`, nicht anfällig für Groß- und Kleinschreibung. Es ist also egal, ob du `'content-type'` oder `'CONTENT-type'` schreibst.

`'Date'` gibt uns Datum und Uhrzeit der Server-Antwort zurück, wie du vielleicht schon vermutet hast. `'content-type'` gibt an, welche Art von Inhalt wir erhalten haben. In unserem Fall ist das ein Text, der mit `'UTF-8'` kodiert ist und HTML-Code enthält. Wirf einen Blick in den Text, indem du das Attribut `my_response.text` ausgibst.

in diesem HTML-Code steckt unsere DAX-Auflistung.
Das sieht jetzt auf den ersten Blick ganz schön wüst aus. Daten in HTML-Form sind aber nicht gänzlich unstrukturiert. Man nennt sie semistrukturiert. Semistrukturierte Daten tragen einen Teil der Strukturinformationen mit sich, anstatt wie eine Tabelle bereits einer allgemeinen Struktur zu unterliegen. Dir sind vielleicht schon weitere semistrukturierte Datenformate über den Weg gelaufen. Dazu gehören beispielsweise [JSON](https://de.wikipedia.org/wiki/JavaScript_Object_Notation) und [XML](https://de.wikipedia.org/wiki/Extensible_Markup_Language). 
Im weiteren Verlauf wirst du sehen, was es heißt, dass HTML einen Teil der Strukturinformationen mit sich trägt.

Wenn man Daten strukturiert, sollte man auf 3 Prinzipien achten, um sicherzustellen, dass die Daten möglichst einfach für automatisierte Auswertungen und Modelle verwendet werden können. Diese Prinzipien werden häufig tidy data principles genannt. Sie lauten wie folgt:

Jede Beobachtung hat eine eigene Zeile
Jede Variable hat eine eigene Spalte
Jeder Wert hat eine eigene Zelle
In diesem Kapitel werden wir öffentlich zugängliche Daten von Webseiten auslesen und diese anschließend in eine strukturierte Form gemäß diesen Prinzipien bringen. Also so, wie wir selbst gerne Daten erhalten würden.

---

# Aufgabe 2: HTML Grundlagen

Wenn die Abfrage erfolgreich vom Server verarbeitet wurde, befindet sich nun der HTML-Code der Webseite im Attribut `my_response.text`. Drucke die ersten 125 Zeichen deines Textes und wirf einen Blick hinein.

Du solltest in etwa folgenden Text erhalten:

`'<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>DAX - Wikipedia</title>\n'`

Wir sehen hier schon die typische Struktur eines HTML-Dokuments. HTML steht für *Hypertext Markup Language* und ist eine Auszeichnungssprache, welche 1992 von der Europäischen Organisation für Kernforschung (CERN) verbreitet wurde. HTML ist fester Bestandteil des *World Wide Web*, eine grundlegende Kenntnis darüber ist also beim *web scraping* hilfreich.

Auszeichnungssprachen wie HTML zeichnen sich dadurch aus, dass man Elemente (z.B. Textabsätze, Bilder oder Animationen) mit Eigenschaften, Zugehörigkeiten und Darstellungsformen anreichern kann. Dies geschieht in der Regel, indem man sie mit sogenannten Tags markiert. Dabei handelt es sich um die Strukturinformationen, die wir in der letzten Übung erwähnt haben.

Jedes HTML-Dokument hat einen festen Aufbau, der wie folgt aussieht:

```html
<!DOCTYPE html>
<html lang="en">
  <head>
    Der Kopf enthält Informationen über das Dokument. Diese werden als eigene Elemente dargestellt.
    <title>Hier steht der Titel</title>
    ...
  </head>
  <body>
    Hier steht der Webseiten-Körper. Er ist das, was im Browser dargestellt wird.
    ...
  </body>
</html>

```
In HTML siehst du oft die Kombination aus einem Wort in spitzen Klammern, Text dazwischen und dem gleichen Wort in spitzen Klammern und einem Schrägstrich `'/'`, z.B. `'<title>DAX - Wikipedia</title>'`. Das sind die Tags. Diese geben dem ganzen Dokument ihre Struktur. Mit der ersten Nennung öffnen sie ein Element, z.B. den Titel. Der folgende Text entspricht dem Wert dieses Elements (z.B. `'DAX - Wikipedia'`). Die Nennung mit einem Schrägstrich (z.B. `'</title>'`) beendet das Element. Innerhalb eines Elements kann man auch neue Elemente anfangen, sie lassen sich also verschachteln. Die Verschachtelung wird im Beispiel des Aufbaus durch die Einrückung signalisiert.

HTML-Tags können nicht nur Text beinhalten, sondern auch Attribute. Diese geben weitere Informationen zum Element. Ein Beispiel hierfür siehst du im `<html>`-Tag. Dieses hat z.B. das Attribut `lang='en'`, welches die Sprache des Dokuments als Englisch spezifiziert.

HTML kennt zahlreiche Tags und noch mehr Attribute, diese hat man in der Regel nicht alle im Kopf. Deshalb bieten dir Browser wie Firefox und Chrome spezielle Tools an, um diese Informationen auf einer Seite anzuzeigen. Bei Firefox handelt es sich um den [*Page Inspector*](https://developer.mozilla.org/en-US/docs/Tools/Page_Inspector) und bei Chrome um die [Entwicklertools](https://developers.google.com/web/tools/chrome-devtools/).

Folgende *Tags* sind häufiger zu finden:
* `<h1>`: Beschreibt Überschriften, steht statt der 1 eine andere Zahl, so wird eine andere Überschriftsebene beschrieben.
* `<p>`: Beschreibt einen Paragrafen.
* `<a>`: Beschreibt einen Hyperlink (*anchor tag*).
* `<div>`: Beschreibt einen Abschnitt im Dokument, wird beispielsweise verwendet, um Textbereiche hervorzuheben.
* `<span>`: Wie `<div>`, aber für kürzere Abschnitte.
* `<img>`: Beschreibt ein Bild.
* `<table>`: Beschreibt eine Tabelle.
* `<tr>`: Beschreibt eine Zeile (*row*) einer Tabelle.
* `<td>`: Beschreibt eine Zelle einer Tabelle.

Attribute helfen oft dabei, die richtigen Elemente zu identifizieren. Vor allem die `<div>`- und `<span>`-Elemente werden teils inflationär verwendet, was das Extrahieren von Informationen schwieriger gestalten kann.

Die Daten, die wir benötigen, befinden sich in einer Tabelle. Wie viele davon weist die Webseite auf? Zähle den entsprechenden Tag.

Tipp: Nutze die `my_string.count()`-Methode. Denke daran, dass Attribute innerhalb der Tags stehen können.

Ich habe 7 Tabellen gefunden (Stand März 2021). Die Zahl kann leicht bei dir abweichen, falls sich die Webseite inzwischen etwas verändert hat. Wir könnten jetzt mit Hilfe von *string*-Methoden alle Tabellen ausfindig machen und deren Inhalte drucken, um herauszufinden, wo die Inhalte stehen, die wir brauchen. Zum Glück müssen wir uns das aber nicht antun. Stattdessen verwenden wir ein Modul, welches uns das Arbeiten mit HTML erleichtert.

---

# Aufgabe 3: Tabelle aus dem HTML DOM parsen

Wie wir gerade gesehen haben, müssen wir uns entlang der HTML-Tags hangeln, wenn wir die Unternehmensdaten aus der Webseite isolieren möchten. Allein mit *string*-Methoden vorzugehen ist sehr mühsam, zumal die *Tags* ineinander verschachtelt sein können. Hier kommen sogenante HTML-Parser ins Spiel. Sie verstehen die Struktur, welche durch die Tags vorgegeben wird und helfen uns dabei, bestimmte Elemente zu suchen. Ein Modul, mit welchem wir das durchführen können, ist `lxml`.

`lxml` ist ein sehr performanter Parser für XML-Dokumente. XML steht für *Extensible Markup Language*. XML und HTML sind eng miteinander verwandt. Der größte Unterschied besteht darin, dass die Tags in HTML vorgegeben sind. `lxml` weist das Submodul `lxml.html` auf, welches speziell für HTML-Dokumente maßgeschneidert ist. Es beinhaltet die Funktion `fromstring()`. Wenden wir diese auf einen HTML-*string* an, so erhalten wir ein Objekt zurück, welches es uns einfacher macht, Elemente im HTML-Code zu suchen.

Importiere `lxml.html`. Wende anschließend `fromstring()` auf den HTML-Code der Webseite an und speichere das Ergebnis als Variable namens `root`.

Wir interessieren uns nur für die `<table>`-Tags. Denn in einer dieser Tabellen stecken die Daten, die wir extrahieren möchten.
`lxml` hat uns mit `root` ein Objekt erzeugt, welches einem sogenannten *element tree* entspricht. Daher kommt der Name `root`. Es ist sozusagen die Wurzel des Datentyps. Das heißt für uns, dass dieses Objekt einige Methoden aufweist, die es uns einfacher machen, gewisse Elemente zu finden. Eine dieser Methoden ist `my_root.iter()`. Sie gibt uns die Möglichkeit mit einer Schleife durch die Elemente des HTML-Dokuments zu iterieren. Wenn wir ihr einen Tag vorgeben, so iterieren wir nur durch Elemente mit diesem Tag.

Iteriere durch alle Tabellen. Überreiche dazu `'table'` an `root.iter()`. Nenne deine Laufvariable `table` und drucke sie bei jeder Iteration.

Tipp: Du kannst dir `root.iter()` so vorstellen, wie `range()`, nur dass du Anstelle von Zahlen Elemente des HTML-Dokuments erhältst.

Du solltest mehrmals so etwas wie `<Element table at 0x1b9fd112368>` erhalten haben. Jede Tabelle ist also ein Element mit dem Tag `table`. Dieses Element beinhaltet alles, was zwischen dem `<table>`- und dem `</table>`-Tag steht. Aber wie finden wir jetzt heraus, welche von den ganzen Tabellen wir brauchen?

Hier hilft uns der [*Page Inspector*](https://developer.mozilla.org/en-US/docs/Tools/Page_Inspector) bzw. die [Entwicklertools](https://developers.google.com/web/tools/chrome-devtools/). Wählen wir damit die gesamte Tabelle innerhalb der [Wikipediaseite](https://en.wikipedia.org/wiki/DAX) aus, sieht das etwa so aus:

![Page Inspector Auswahl](https://raw.githubusercontent.com/AlexHiesch/DAX/master/02_01_02_pic2.png)

Hier sehen wir, dass der `<table>`-Tag einige Attribute beinhaltet. Bei mir sieht er wie folgt aus `<table class="wikitable sortable jquery-tablesorter" style="text-align: center; font-size: 100%;" id="constituents" cellspacing="2" cellpadding="2">`. 
Damit kennen wir die Attribute der benötigten Tabelle und können abgleichen, welche der Tabellen diese aufweist.

Iteriere dazu wieder durch alle Tabellen. Drucke diesmal aber das Attribut `my_element.attrib`. Alle Elemente von `root` weisen es auf. Dabei handelt es sich um ein *dictionary* mit den Attributen des jeweiligen Tags.

Wir können hier sehen, dass die Attribute teils sehr unterschiedlich sind. Das hilft uns beim *web scraping* dabei, die richtigen Elemente auszuwählen. Schlecht gemachte Webseiten machen diese Aufgabe sehr schwierig. Da ist es dann wichtig, die richtige Kombination aus Attributen zu finden. Gute Webseiten machen es uns einfach, indem sie z.B. das `id`-Attribut nutzen. Bei HTML darf jede `id` nur einmal vorkommen, so dass die Elemente eindeutig zu unterscheiden sind.

In unserem Fall benötigen wir die Tabelle mit der `id` `'constituents'`. `lxml.html` hat `root` mit der Methode `my_root.get_element_by_id()` ausgestattet. Der Name ist hier Programm. Die Methode ermöglicht es uns, ein ganz bestimmtes Element nur an Hand der `id` auszuwählen. Wähle das Element mit der `id` `'constituents'` und speichere es unter dem Variablennamen `table`.

**Achtung:** Wenn du statt `html` `etree` nutzt, wie man das häufig online findet, dann sind sehr viele Befehle identisch, `etree` verfügt jedoch nicht über einige Befehle, die das Arbeiten mit HMTL besonders leicht machen. Dazu gehört beispielsweise `my_root.get_element_by_id()`.

Nun haben wir die Tabelle als eigene Variable. Jetzt müssen wir nur noch die Daten herausziehen. Dabei hilft es uns, wenn wir die typische Tabellenstruktur in HTML kennen. Diese könnten wir uns mit dem *Page Inspector* bzw. den Entwicklerwerkzeugen erschließen. Eine Tabelle hat in HTML bis zu 3 Abschnitte: Tabellenkopf (`<thead>`), Tabellenkörper (`<tbody>`) und Fußzeile (`<tfoot>`). Jeder dieser Abschnitte besteht aus Zeilen (`<tr>`), die wiederum aus Header-Zellen (`<th>`) oder Daten-Zellen (`<td>`) bestehen. Das Ganze kann man sich etwa so vorstellen:

```html
<table>
  <thead>
    <th>...</th>
    ...
  </thead>
  <tbody>
    <tr>
      <td>...</td>
      ...
    </tr>
    ...
  </tbody>
  <tfoot>
    <tr>
      <td>...</td>
      ...
    </tr>
  </tfoot>
</table>
```

Wenn wir es mit einem `DataFrame` vergleichen, dann stecken in den `<th>`-Elementen die Spaltennamen und in den `<td>`-Elementen die Daten. Wofür wir uns nun also interessieren, sind die Inhalte der einzelnen Zellen. Ohne das richtige Werkzeug ist es aber gar nicht so einfach, sie auszulesen. Denn jede Zelle kann weitere Tags enthalten. `lxml.html` hilft uns aber auch hier weiter.

Zuerst einmal verfügen die Elemente über die Methode `my_element.text_content()`. Sie gibt uns sämtlichen Text zwischen dem Öffnungstag und dem Schlusstag des jeweiligen Elements und aller Elemente, die sich ansonsten noch darin befinden. Probiere es aus und drucke die Textinhalte aller Elemente innerhalb von `table`.

Wir erhalten einen *string*, der sehr oft die Zeichenfolge `'\n'` enthält, welcher für Zeilenumbrüche steht. `my_element.text_content()` trennt nämlich die Werte der einzelnen Tags durch Zeilenumbrüche. `lxml.html` hilft uns weiterhin damit, dass wir `table` genauso durchsuchen können wie `root`. Jedes Element stellt einen Unterbaum unserer Struktur dar und bietet uns wieder dieselben Methoden. 

In unserem Fall heißt das, dass wir uns den Inhalt jeder Zeile (`<tr>`) ausgeben lassen können und damit sofort die Inhalte der zugehörigen Zellen erhalten. Da es sich um *strings* handelt, können wir die `my_string.split()`-Methode verwenden, um die Werte voneinander zu trennen.

Iteriere durch alle Zeilenelemente (`<tr>`) von `table`. Nutze `my_string.split()` innerhalb der Schleife, um die Textinhalte der Zeilenelemente entlang der Zeilenumbrüche in Listen zu trennen. Speichere diese Listen in einer übergeordneten Liste namens `table_list`. Drucke `table_list` nach der Schleife.

Überprüfe nun die Anzahl der Elemente in jeder Unterliste von `table_list`.

Bei mir haben die Unterlisten von `table_list` jeweils die Länge 9. Sie sind also alle gleich lang. Das macht es uns einfach, sie in eine Tabelle zu übertragen, die wir dann speichern können. Unsere Tabelle wird entsprechend 9 Spalten aufweisen. 

---

# Aufgabe 4: Tabelle in einen DateFrame überführen

Erzeuge aus `table_list` einen `DataFrame` namens `df_dax`. Die erste Liste von `table_list` sollte für die Spaltennamen verwendet werden. Drucke anschließend die Anzahl der Zeilen und Spalten sowie die ersten 5 Zeilen von `df_dax`.

Mein `df_dax` sieht so aus:

![Erste Zeilen von df_dax](https://raw.githubusercontent.com/AlexHiesch/DAX/master/02_01_02_pic1.png)

Die Inhalte können bei dir abweichen, da sie sich inzwischen vielleicht verändert haben.

Beim Betrachten von `df_dax` fällt mir auf, dass nur 6 der 9 Spalten angezeigt werden. Ich habe also 3 komplett leere Spalten. Das liegt vor allem daran, dass Links und Bilder ohne Textinhalt zu Zeilenumbrüchen ohne Inhalte geführt haben. So haben wir beispielsweise nicht die Logos der Unternehmen, die aus Link und Bild bestehen. Lösche die Spalten, die als Spaltennamen nur einen leeren *string* (`''`) haben und gib alle 30 Unternehmen aus.

---

# Aufgabe 5: DataFrame bereinigen

Am besten passen wir noch die Datentypen von `df_dax` an. So stellen wir sicher, dass die Daten so sind, dass wir sie in Zukunft leicht nutzen können.
Welche Datentypen liegen uns gerade vor?

Bei mir haben alle Spalten den Typ `object`. Das war zu erwarten, denn wir haben nur Listen aus *strings* verwendet.

In der Spalte mit der Mitarbeiteranzahl befinden sich noch Jahreszahlen. Da wir die nicht brauchen, entfernen wir sie kurz. Dazu benutzen wir einen regulären Ausdruck.

In [ ]:
df_dax.loc[:, 'Employees']=df_dax.loc[:, 'Employees'].str.replace(r'\(\d\d\d\d\)','')

Wandle die Spalten, die keinen Text enthalten, in numerische Datentypen um. Überprüfe anschließend die Datentypen.

Tipp: Nutze die Funktion `pd.to_numeric()`. Wenn einzelne Werte nicht numerisch sind (da sie z.B. fehlende Werte repräsentieren) kannst du folgenden Parameter nutzen `errors='coerce'`.
Wenn Spalten Kommata als Tausenderzeichen verwenden, dann solltest du diese durch eine *string*-Methode entfernen.

---

# Aufgabe 6: Daten anreichern über Web Crawling

Nun werfen wir einen Blick in die Links, die in der Tabelle stecken. Links befinden sich in den `<a>`-Tags (a wie *anchor*). Drucke die Attribute aller Links in der Tabelle aus.

Aus den Attributen ist zu sehen, dass wir verschiedene Arten von Links haben. Einige Links verweisen auf Inhalte der Wikipediaseite. Diese erkennen wir daran, dass ihr `href`-Attribut mit `'/wiki/'` beginnt. `href` steht für *hypertext reference* und bezeichnet die Addresse, auf die verlinkt wird.
Die Links unterteilen sich in 2 Kategorien. Die eine Kategorie bindet Bilddateien (die Logos der Unternehmen) ein, sie lässt sich durch die `class` `'image'` identifizieren. Die andere Kategorie führt zu den Unternehmensseiten innerhalb von Wikipedia. Die zugehörigen Links weisen nur das Attribut `title` auf. Weiterhin gibt es noch Links, die auf externe Inhalte weisen, in unserem Fall Webseiten der Frankfurter Börse. Diese Links werden mit der Klasse `'external text'` ausgezeichnet. Außerdem haben sie das Attribut `rel="nofollow"`. Dies bedeutet lediglich, dass diese Links keinen Einfluss auf das Ranking bei Suchergebnissen haben sollen. Dann gibt es noch Links, die mit einer Raute `'#'` beginnen. Sie zeigen auf andere Abschnitte innerhalb der gleichen Webseite.

Die Links, für die wir uns interessieren, sind die, welche auf die Unternehmenswebseiten innerhalb von Wikipedia verweisen. Erzeuge eine Liste namens `links_wiki`. Fülle diese mit den Werten des `href`-Attributs der entsprechenden Links. Drucke anschließend die ersten 5 Webadressen in `links_wiki`.

Tipp: Die gesuchten Links enthalten nicht das Attribut `class`. Mit `if not 'key' in my_dict` kann überprüft werden, ob ein gewisser *key* in einem *dictionary* enthalten ist.

Bei mir sind die Links `'/wiki/Prime_Standard'` und `'#endnote_1'` in `links_wiki` enthalten. Diese führen nicht auf Unternehmensseiten, deshalb brauchen wir sie nicht. Wenn du auch Links hast, die nicht auf Unternehmensseiten führen, kannst du sie in folgender Codezelle löschen:

In [ ]:
links_wiki.remove('/wiki/Prime_Standard')
links_wiki.remove('#endnote_1')
links_wiki[:5]

Nun haben wir einige Links in einer Liste. Diesen Links werden wir folgen und die jeweiligen Seiten automatisch auslesen. Das Durchsuchen ganzer Webseiten-Strukturen nennt man *web crawling*. Dabei gibt es einige Dinge zu beachten. Zum einen möchten einige Webseitenbetreiber nicht, dass ihre Seiten ausgelesen werden. Zum anderen darf man nicht zu viele Anfragen in zu kurzer Zeit stellen. Das kann dazu führen, dass die Server überlastet werden und die Webseite nur langsam reagiert. Webseiten auf diese Art auszubremsen, ist allgemein nicht erwünscht. Davor schützen sich die Webseiten, indem sie die IP-Adressen von Computern blocken, die zu viele Zugriffe anfragen. Teilweise werden auch Fallen für *web crawler* gestellt. Diese sogenannten *honeypots* sind Links, welche auf der Browser-Oberfläche nicht angezeigt werden. *Web crawler*, die einfach allen Links folgen, nutzen sie aber und werden daraufhin blockiert.

Am besten beachtet man die *robots.txt* der Webseite, die man auslesen möchte. Sie zeigt, ob die Webseite automatisiert ausgelesen werden darf oder das nicht erwünscht ist. Ihr Addresspfad entspricht immer der Basisadresse der Webseite, an die */robots.txt* angehängt wird. In unserem Fall ist das also *https://en.wikipedia.org/robots.txt*.

Wenn du einen Blick in die *robots.txt* von Wikipedia wirfst, siehst du folgende Textausschnitte:
```
User-agent: Zao
Disallow: /
```
Viele seriöse *web crawler* überreichen bei einer HTTP-Abfrage sozusagen ihren Namen als `User-agent`. In diesem Beispiel ist es so, dass Wikipedia nicht möchte, dass der *web crawler* namens `Zao` ihre Seite durchsucht. Deshalb steht dort `Disallow: /`. Das bedeutet, dass alle Adressen ab der Basisadresse nicht erlaubt sind. Steht neben `Disallow` nichts, dann sind alle Unterseiten erlaubt.

An welche Anweisungen müssen wir uns halten? `requests` identifiziert sich standardmäßig mit dem Modulnamen und der zugehörigen Version. In meinem Fall ist das `User-agent: python-requests/x.xx.x`. In der *robots.txt* ist dieser Name nicht speziell angegeben. Alle *web crawler*, die nicht explizit aufgeführt sind, sollten sich an die Anweisung halten, die bei `User-agent: *` aufgeführt sind. Der Stern steht für alle sonstigen Namen. Bei mir steht hier:

```
User-agent: *
Allow: /w/api.php?action=mobileview&
Allow: /w/load.php?
Allow: /api/rest_v1/?doc
Disallow: /w/
Disallow: /api/
Disallow: /trap/
Disallow: /wiki/Special:
Disallow: /wiki/Spezial:
Disallow: /wiki/Spesial:
Disallow: /wiki/Special%3A
Disallow: /wiki/Spezial%3A
Disallow: /wiki/Spesial%3A
```

Hier stehen einige Pfade, die explizit erlaubt und nicht erlaubt sind. Alle übrigen sind erlaubt. Unsere Link-Adressen stehen hier nicht, wir können also getrost weitermachen.

Wikipedia hat eine sehr konsistente Struktur. So sind die Unternehmensseiten ähnlich aufgebaut. Besuche die Seite *https://en.wikipedia.org/wiki/Adidas*. Dort findest du auf der rechten Seite eine Tabelle mit einigen Kennzahlen, wie dem Umsatz (*Revenue*). Wie lässt sich diese Tabelle identifizieren? Überprüfe sie mit dem *Page Inspector* oder den Entwicklertools oder ähnlichen Werkzeugen deines Browsers. Bei mir hat die Tabelle das Attribut `class="infobox vcard"`. Die anderen Seiten haben eine ähnliche Tabelle mit der gleichen Klasse. Diese werden wir nun nutzen, um die Tabelle auszulesen.

Verbinde dich nun mit der ersten Seite, die in `links_wiki` gegeben ist. Achte darauf, dass du die Basisadresse `'https://en.wikipedia.org'` voranstellen musst. Speichere diese als `base_url`. Speichere außerdem die Antwort als `response` und überprüfe den Statuscode.

Wenn die Abfrage erfolgreich war, dann kannst du den Inhalt der Seite nun mit `lxml.html` umwandeln, um sie anschließend gut durchsuchen zu können. Nenne die entstehende Variable `root`.

`lxml.html` stellt uns wieder eine Methode zur Verfügung, die es uns erleichtert, Elemente mit ganz speziellen Klassen ausfindig zu machen: `my_element.find_class()`. Dieser Methode überreichen wir eine Klasse und erhalten eine Liste mit Elementen, die diese Klasse aufweisen, zurück.

Nutze `my_element.find_class()` zusammen mit `'infobox vcard'`, um die gesuchte Tabelle aus `root` zu extrahieren. Denke daran, dass `my_element.find_class()` eine ganz normale Liste zurückgibt. Da die Webseite aber nur eine Tabelle mit der Klasse `'infobox vcard'` aufweist, hat diese Liste nur ein Element. Wähle es explizit aus und speichere dieses als `'table'`.

Wir sind vorallem an den finanziellen Kennzahlen interessiert. Das sind in dieser Tabelle die Werte von *Revenue*, *Operating income*, *Net income*, *Total assets* und *Total equity*. Am besten lesen zuerst einmal alle Werte aus und nehmen danach die Werte, die wir wirklich haben wollen. Wenn wir den HTML-Code der Tabelle im Browser genauer ansehen, können wir erkennen, dass die Zeilen der Tabelle jeweils aus einen `<th>`-Tag (dem Namen, z.B. *Revenue*) und einem `<td>`-Tag mit dem entsprechenden Wert bestehen. Einige `<td>`-Tags haben das Attribut `colspan="2"`. Das sind z.B. Logos. Diese Zellen passen nicht so recht in die Struktur der Tabelle, deshalb lesen wir sie nicht mit ein.

Führe die folgende Zelle aus, um mit einer *list comprehension* alle Inhalte der `<td>`-Tags in der Liste `table_values` zu speichern. Durch eine `if`-Abfrage werden nur die Zellen beachten, die nicht das Attribut `colspan="2"` aufweisen.

In [ ]:
table_values = [element.text_content()
                for element in table.iter('td')
                if not ('colspan' in element.attrib and element.attrib['colspan']=='2')] 

Speichere nun die Inhalte der `<th>`-Tags in der Liste `table_names`. Überprüfe die Länge von `table_names` und `table_values`. Du brauchst dabei nicht auf das Attribut `colspan="2"` zu achten, da es nicht vorkommt.

Nun werden wir dieselben Schritte für alle Unternehmensseiten aus `links_wiki` durchführen. Allerdings möchten wir nicht aufgrund zu vieler Anfragen in zu kurzer Zeit geblockt werden. Deshalb müssen wir zwischen unseren Anfragen eine bestimmte Zeit lang warten. Das können wir mit dem `time`-Modul umsetzen. Importiere es und nutze die Funktion `time.sleep(5)`. 



Web crawler* von Yahoo und Microsoft halten sich an eine Vorgabe in der *robots.txt*. Diese Vorgabe wird durch `Crawl-delay` ausgedrückt.  Wenn keine Zeit angegeben ist, empfiehlt sich das fünffache von der Serverantwort. Diese bekommen wir über `my_response.elapsed.total_seconds()` heraus. Was wäre dies bei dir multipliziert mit 5?

Da die Unternehmensseiten alle eine ähnliche Struktur in der Tabelle aufweisen, können wir nun mit einer Schleife durch `links_wiki` iterieren und die Werte der Tabelle mit Klasse `'infobox vcard'` auslesen. Erzeuge zuerst zwei Listen namens `table_names_all` und `table_values_all`. Diese sollen am Ende Listen mit den Werten der `<th>`- bzw. der `<td>`-Tags enthalten.

Deine Schleife sollte folgende Schritte ausführen:
1. Verbinden mit der jeweiligen Seite (denke dabei daran, auch `base_url` zu nutzen)
2. Überprüfen des Statuscodes
3. Parsen des HTML-Inhalts
4. Die Tabelle mit der Klasse `'infobox vcard'` auswählen
5. Die Inhalte aller `<th>`-Tags in der Liste `table_names` speichern
6. Die Inhalte aller `<td>`-Tags in der Liste `table_values` speichern, falls sie nicht das Attribut `colspan="2"` aufweisen
7. `table_names` und `table_values` in `table_names_all` bzw. `table_values_all` speichern
8. 5 Mal die Antwortzeit abwarten, bevor die nächste Anfrage gestellt wird

Tipp: Orientiere dich an dem Code, den wir in den letzten 8 Codezellen benutzt haben. Da immer wieder gewartet wird, kann es eine Weile dauern, bis die Schleife durchgelaufen ist.

Jetzt können wir die Daten schon fast in einen `DataFrame` packen. Das geht am besten, wenn wir eine Liste aus *dictionaries* nutzen. Die *keys* entsprechen dann immer den Spaltennamen, welche in  `table_names_all` stecken und die zugehörigen *values* sind in `table_values_all`. Um ein *dictionary* zu erzeugen, müssen wir `dict()` eine Liste aus *key*-*value*-Tupeln überreichen. Wenn wir die *keys* und die *values* in getrennten Listen haben, können wir sie direkt mit `zip()` verbinden. 

In folgender Zelle wird eine Liste `company_dicts_list` erzeugt. Diese wird dann nach und nach mit *dictionaries* gefüllt. Führe sie aus.

In [ ]:
company_dicts_list = []  
for table_names, table_values in zip(table_names_all, table_values_all):  
    company_dict = dict(zip(table_names, table_values)) 
    company_dicts_list.append(company_dict) 

Aus einer Liste mit *dictionaries* kann `pandas` direkt einen `DataFrame` erzeugen. Das einzige, was uns dann noch fehlt, sind die Zeilennamen. Hier bieten sich die Unternehmensnamen an. Diese stecken in den Links von `links_wiki`. Das Ende der *strings* darin besteht immer aus `'/CompanyName'`. Erzeuge eine neue Liste namens `company_names`. Fülle sie mit den Unternehmensnamen.

Tipp: Nutze die Methode `my_string.split()`.

Nun können wir die Daten aus den Unternehmenswebseiten endlich in einen `DataFrame` übertragen. Nutze `company_dicts_list` als Werte und `company_names` als Zeilennamen. Denke daran `pandas` zu importieren. Speichere den `DataFrame` unter dem Namen `df_company` und drucke die ersten 5 Zeilen.

Ich habe sehr viele fehlende Werte. Das ist aber ganz normal, denn obwohl die Tabellen auf Wikipedia eine ähnliche Struktur haben, sind die gezeigten Informationen unterschiedlich. Beim *web crawling* mehrerer Internetseiten ist es nur sehr selten der Fall, dass sich die Informationen so decken, dass keine fehlenden Werte entstehen.

---

# Aufgabe 7: Crawldaten mit Regexp bereinigen

Da wir uns nur für die finanziellen Kennzahlen interessieren, konzentrieren wir uns auf die Spalten `['Operating income', 'Net income', 'Revenue', 'Total assets', 'Total equity']`. Entferne die übrigen Spalten. Drucke anschließend die ersten 5 Zeilen, um das zu überprüfen. 

Die Finanzkennzahlen sind als Texte gegeben. Sie bestehen aus dem eigentlichen Betrag, mit Währungssymbol und einem Wort (z.B. `'billion'`), sowie einer Jahresangabe und einem Vermerk (z.B. `'[1]'`). Das entspricht nicht den *tidy data principles*. Deshalb werden wir jetzt aus jeder Spalte zwei Spalten erzeugen, die den Betrag bzw. das Jahr enthalten. Dazu nutzen wir *regular expressions*. Die Wichtigsten kannst du den folgenden Tabellen entnehmen.



### Verallgemeinerungen
| Ausdruck   | Bedeutung                                           |
| ---        | ---                                                 |
| .          | beliebiges Zeichen (außer Zeilenumbruch)            |
| \d         | Ziffer                                              |
| \w         | Buchstabe, Ziffer oder Unterstrich                  |
| \s         | *Whitespace*: Leerzeichen, Tabulator, Zeilenumbruch |
| \D, \W, \S | Negation von \d, \w und \s                          |

### Wiederholungen
| Ausdruck   | Bedeutung                                                                        |
| ---        | ---                                                                              |
| {min, max} | Wiederholung des voranstehenden Ausdrucks mindestens min-Mal und maximal max-Mal |
| {3}        | Wiederholung des voranstehenden Ausdrucks genau 3 Mal                            |
| +          | Wiederholung des voranstehenden Ausdrucks mindestens 1-Mal                       |
| *          | Wiederholung des voranstehenden Ausdrucks mindestens 0-Mal                       |

### Kombinationen
| Ausdruck   | Bedeutung                                                                                                      |
| ---        | ---                                                                                                            |
| ...\|...   | Oder-Verknüpfung: Der Ausdruck links oder rechts von \| wird gesucht                                           |
| [...]      | Oder-Verknüpfung: Ein Ausdruck innerhalb der Klammern wird gesucht                                             |
| [a-z]      | Bis-Verknüpfung: Nur ein Kleinbuchstabe zwischen a und z wird gesucht                                          |
| [A-Z0-9]   | Bis-Oder-Verknüpfung: Ein Großbuchstabe zwischen A und Z oder eine Ziffer zwischen 0 und 9 wird gesucht        |
| [^...]     | Außer: Nur Ausdrücke, die nicht in den Klammern aufgeführt sind werden gesucht                                 |
| (...)      | *capture group*: Alle Ausdrücke innerhalb der Klammern werden gemeinsam gesucht und bilden ein eigenständiges Ergebnis |
| ?          | Optionalität: Der voranstehende Ausdruck darf genau 0-Mal oder 1-Mal zutreffen                                 |
| \          | *Escape*: Das folgende Zeichen wird so wie es ist gesucht (z.B. \\. sucht einen Punkt)                          |

### Grenzen
| Ausdruck   | Bedeutung                                          |
| ---        | ---                                                |
| ^          | Der Anfang des *string* wird gesucht               |
| \$          | Das Ende des *string* wird gesucht                |
| \\b         | Der Anfang oder das Ende eines Worts wird gesucht |

Glücklicherweise bietet uns `pandas` die Möglichkeit, *regular expressions* zu nutzen. Dafür können wir die Methode `my_series.str.extract()` nutzen. Sie erwartet eine *regex* mit mindestens einer *capturing group*. Diese entstehen durch runde Klammern um das Suchmuster. Wenn du also nach einer Ziffer suchst, dann sieht die *capturing group* wie folgt aus: `r'(\d)'`. Trifft das Suchmuster öfter zu, so wird nur das erste Ergebnis ausgegeben.

**Achtung:** `my_series.str.extract()` gibt standardmäßig einen `DataFrame` zurück. Dieser enthält eine Spalte für jede *capturing group* im Suchmuster. Das führt dazu, dass sich die erhaltene Spalte nicht ohne weitere Funktionen hinzufügen lässt. Nutze deshalb den Parameter `expand=False`, damit sorgst du dafür, dass eine `Series` zurückgegeben wird.

Jetzt fehlt uns noch der Betrag in einer eigenen Spalte. Diesen zu extrahieren, ist etwas kniffeliger. Denn am besten nehmen wir auch die Wörter (wie `'billion'`) mit, um sie später durch die entsprechenden Zahlen zu ersetzen. Auf das Währungssymbol können wir erst einmal verzichten. Wir sollten aber im Kopf behalten, dass nicht alle Beträge unbedingt in Euro angegeben sind. Speichere das Ergebnis als neue Spalte namens `Total assets value`.

Tipps: Mit eckigen Klammern kannst du angeben, dass nur eines der darin enthaltenen Zeichen zutreffen muss. So kannst du z.B. mit `r'[\d.,]+'` alle Zahlen, die durch Punkte oder Kommas getrennt sind erhalten. Möchte man nur Buchstaben erhalten, so benötigt man auch eckige Klammern. Man muss aber nicht jeden Buchstaben einzeln einfügen, sondern kann einen Bindestrich als "bis" verwenden. `r'[a-zA-Z]'` trifft beispielsweise auf alle Klein- und Großbuchstaben zwischen a und z zu. Mit `r'?'` kannst du den vorangehenden Teil des Suchmusters optional gestalten. Wenn du dir nicht sicher bist, ob zwischen Zahl und Wort ein Leerzeichen steht, kannst du durch `'\s?'` beide Fälle abdecken.

ch habe nun in der Spalte `'Total assets value'` Zahlen und Wörter, die zur Zahl gehören. Die Wörter sind in meinem Fall `'million'`, `'billion'`, `'trillion'` und `'bn'`.
Damit wir die Spalte in Zahlen umwandeln können, sollten wir diese Werte ersetzen. Das englische 'billion' (kurz 'bn') ist im Deutschen die Milliarde und 'trillion' ist die Billion. Sie können wir durch die wissenschaftliche Notation 1e9 bzw. 1e12 ersetzen, dann lassen sich die Zahlen problemlos in Fließkommawerte umwandeln. Bei mir gibt es noch eine Zahl, die ein Komma als Trennzeichen benutzt. Dieses ersetze ich durch einen Punkt. Nicht nur my_series.str.extract() kann mit regex umgehen, sondern auch my_series.str.replace(). So nutze ich z.B. den Code r'\s', um das Leerzeichen zu ersetzen.

Führe die folgende Codezelle aus, um das Komma und die Wörter `'million'`, `'billion'`, `'trillion'` und `'bn'` ersetzen. Sollten bei dir noch weitere Zahlenwerte vorkommen, kannst du die entsprechende Ersetzung hier einfügen. Am Ende der Zelle wird die Spalte in eine `float`-Spalte umgewandelt.

Bisher haben wir die Daten aus der Spalte `'Total assets'` in eine Spalte mit der Jahresangabe und eine Spalte mit dem Betrag aufgespaltet. Dabei haben wir folgende Schritte durchgeführt:
* Extrahieren der Jahreszahl mit *regex* in eine neue Spalte
* Extrahieren des Betrags und des zugehörigen Zahlenworts in eine neue Spalte
* Umwandeln des Zahlenworts in die wissenschaftliche Notation
* Ersetzen von Dezimalkomma durch Dezimalpunkt
* Umwandeln der Spalten in ein Zahlenformat


Das müssen wir jetzt noch für die Spalten `['Operating income', 'Net income', 'Revenue', 'Total equity']` wiederholen. Es empfiehlt sich mit einer Schleife zu arbeiten.

Tipp: Eine neuer Spaltenname lässt sich durch Verknüpfung zweier *strings* umsetzen. Angenommen der Spaltenname `'Total assets'` steckt in der Variable `col`, so erzeugt `df_company.loc[:, col+' year']` den Spaltennamen `'Total assets year'`

Mein `DataFrame` besteht nun aus 26 Zeilen und 15 Spalten. Entferne nun unsere Anfangsspalten `['Operating income', 'Net income', 'Revenue', 'Total assets', 'Total equity']`, dann sind wir fast fertig. Drucke dann die ersten 5 Zeilen.

---

# Aufgabe 8: Daten anreichern über API

Im Börsenumfeld sind APIs für Kursdaten entweder stark eingeschränkt wieviele Requests getätigt werden darf oder erst Recht nur kostenpflichtig.

Eine Ausnahme stellt hier das Python Modul [yfiance](https://pypi.org/project/yfinance/) dar.

Installiere es über `!pip install yfinance` und importiere es anschließend

Das Modul greift direkt Yahoo Finance ab, ohne sich authentifizieren zu müssen, wie es sonst eigentlich üblich ist bei API Calls über OAuth Tokens oder Keys.

Historische Kursdaten erhält man über:

`yf.download("<Ticker>", start="yyyy-mm-dd", end="yyyy-mm-dd")`

Grundsätzliche Informationen erhält man über:

`yf.Ticker("<Ticker>")`

Auf das Ticker-Objekt lassen sich dann sämtliche Attribute abrufen wie z.B. `info` für allgemeine Informationen, `history(period="max"` den historischen Höchstpreis, `financials` für die Finanzkennzahlen uvm.

Probier mal die Adidas-Kurse von diesem Monat abzurfen

Plote den Kurs

Und nun schau dir die allgemeinen Informationen an zur Adidas-Aktie

Ergänze den DAX-DataFrame bitte um den Dividentenrendite `dividendYield`

Plote die Dividende über alle Aktien